In [3]:
from sentinelsat import SentinelAPI, make_path_filter # for downloading data
import geopandas as gpd # for reading shapefiles
import folium  # library to create interactive maps
from shapely.geometry import MultiPolygon, Polygon
import os 
import rasterio as rio
from rasterio.mask import mask
import zipfile
import rasterio
import json
# read credentials.json
with open('credentials.json') as f:
    credentials = json.load(f)

user = credentials['SENTINEL_USER']
password = credentials['SENTINEL_PASSWORD']
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
nyc_geo = gpd.read_file('data/layers/POLYGON.shp')

In [5]:
m = folium.Map([40.730610, -73.935242], zoom_start=10) # New York City

folium.GeoJson(nyc_geo).add_to(m)
# visualize the map
m

In [6]:
footprint = None
for i in nyc_geo['geometry']:
    footprint = i

In [7]:
def get_bands(imagery_folders):
    for imagery in imagery_folders:
        l2a_folder = os.listdir(f'{imagery}/GRANULE')[0]
        r10_path = f"{imagery}/GRANULE/{l2a_folder}/IMG_DATA/R10m"
        for band in os.listdir(r10_path):
        # extract bands 2,3, and 4 from r10:

            if 'B02' in band:
                b2 = rio.open(f"{r10_path}/{band}")
            elif 'B03' in band:
                b3 = rio.open(f"{r10_path}/{band}")
            elif 'B04' in band:
                b4 = rio.open(f"{r10_path}/{band}")

    return b2, b3, b4

In [8]:
image_path_2 = 'S2A_MSIL2A_20220825T155151_N0400_R011_T18TWL_20220825T220800.SAFE'
image_path_1 = 'S2B_MSIL2A_20220316T154909_N0400_R054_T18TWL_20220316T205909.SAFE'

In [9]:
b2, b3, b4 = get_bands(image_path_1)


FileNotFoundError: [Errno 2] No such file or directory: 'S/GRANULE'

In [ ]:
for product_id in products_gdf_sorted.index:
    is_online = api.is_online(product_id)
    if is_online:
        downloaded_artifact = api.download(product_id)
    else:
        continue
    zip_file = downloaded_artifact['path']

    try:
        with zipfile.ZipFile(zip_file) as z:
            z.extractall()
            print("Extracted all")
            os.remove(zip_file)
    except:
        print("Invalid file")
    for f in os.listdir():

    imagery_folders = downloaded_artifact.replace('.zip','.SAFE')
    b2, b3, b4 = get_bands(imagery_folders)
    # this will take a while on the Nano..
    with rio.open('RGB.tiff','w',driver='Gtiff', width=b4.width, height=b4.height, 
                count=3,crs=b4.crs,transform=b4.transform, dtype=b4.dtypes[0]) as rgb:
        rgb.write(b2.read(1),1) 
        rgb.write(b3.read(1),2) 
        rgb.write(b4.read(1),3) 
        rgb.close()

    with rio.open("RGB.tiff") as src:
        out_meta = src.meta.copy()
        nyc_geo_projected = nyc_geo.to_crs(out_meta['crs'].data)
        out_image, out_transform = mask(src, nyc_geo_projected.geometry,crop=True) #TODO: note that projected has to be in the geo from out_meta. 
        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform})
        
    with rasterio.open(f"{downloaded_artifact['title']}.tiff", "w", **out_meta) as dest:
        dest.write(out_image)

    os.remove("RGB.tiff")

    try:
        os.remove("RGB.tiff")
        os.remove(downloaded_artifact['path'])
    except Exception as e:
        pass
    plt.figure(figsize=(5,10))    
    plt.title(downloaded_artifact['title'])
    plt.imshow(out_image[0],cmap='gray')

In [ ]:
rasters = []
for f in os.listdir():
    if '.tiff' in f:
        print(f)
        src = rio.open(f).read()
        rasters.append(src)


In [ ]:
!pip install scikit-image

In [ ]:
first = rasters[0]
second = rasters[1]

In [ ]:
img_avg = (first.mean(axis=0) - second.mean(axis=0))

In [ ]:
np.argmax(img_avg,keepdims=True)

In [ ]:
import numpy as np

In [ ]:

max_x, max_y = np.unravel_index(np.argmax(img_avg), img_avg.shape)


In [ ]:
max_x, max_y

In [ ]:
plt.imshow(img_avg[max_x-32:max_x+32,max_y-32:max_y+32],cmap='pink')

In [ ]:
plt.imshow(first[0,max_x-100:max_x+100,max_y-60:max_y+60],cmap='pink')

In [ ]:
plt.imshow(second[0,max_x-100:max_x+100,max_y-60:max_y+60],cmap='pink')

In [ ]:

plt.imshow(first[0],cmap='pink')

In [ ]:
plt.imshow(rasters[1][0] - rasters[1][0] ,cmap='pink')

In [ ]:
import cv2

In [ ]:
(score, diff) = structural_similarity(first.mean(axis=0), second.mean(axis=0), full=True)

In [ ]:
diff[diff > .25] = 1

In [ ]:
diff = (diff * 255).astype("uint8")
diff_box = cv2.merge([diff, diff, diff])

In [ ]:
(score, diff) = structural_similarity(first.mean(axis=0), second.mean(axis=0), full=True)

diff[diff > .75] = 1

diff = (diff * 255).astype("uint8")
diff_box = cv2.merge([diff, diff, diff])

# Threshold the difference image, followed by finding contours to
# obtain the regions of the two input images that differ
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

mask = np.zeros(img_avg.shape, dtype='uint8')
for c in contours:
    area = cv2.contourArea(c)
    if 2000 < area < 4000:
        cv2.drawContours(mask, [c], 0, (255,255,255), -1)
        break
plt.figure(figsize=(55,25))
plt.imshow(second.mean(axis=0))
plt.imshow(mask,alpha=.05)


In [ ]:
contour_y, contour_x = c[1][0]

In [ ]:
contour_y

In [ ]:
plt.imshow(first[0][contour_x - 32:contour_x+32, contour_y - 32:contour_y+32],cmap='pink')

In [ ]:
plt.imshow(second[0][contour_x - 32:contour_x+32, contour_y - 32:contour_y+32],cmap='pink')

In [ ]:
first.shape

In [ ]:
mask

In [ ]:
mask = np.zeros(img_avg.shape, dtype='uint8')
for c in contours:
    area = cv2.contourArea(c)
    if 1000 < area < 4000:
        cv2.drawContours(mask, [c], 0, (255,255,255), -1)

plt.imshow(mask)


In [ ]:
plt.imshow(mask)